<a href="https://colab.research.google.com/github/arjun-593/IITISoC-23-IVR1-LaneDetection-using-LimitedComputationPower/blob/main/NonDL_Lane_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [27]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
import math

In [28]:
def convert_to_gray(img):
    return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)


def edge_detection(img, low_threshold, high_threshold):
    """Applies the Canny transform"""
    return cv2.Canny(img, low_threshold, high_threshold)


def gaussian_blur(img, kernel_size):
    """Applies a Gaussian Noise kernel"""
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)



def region_of_interest(img, vertices):

    #defining a blank mask to start with
    mask = np.zeros_like(img)

    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255


    cv2.fillPoly(mask, vertices, ignore_mask_color)

    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image

In [29]:
def draw_lines(img, lines, color=[255, 0, 0], thickness=10):
  for line in lines:
        for x1,y1,x2,y2 in line:
            cv2.line(img, (x1, y1), (x2, y2), color, thickness)


def assign_slopes_intercepts(image,lines):

    img = image.copy()
    poly_vertices = []
    order = [0,1,3,2]

    left_lines = [] # Like /
    right_lines = [] # Like \
    for line in lines:
        for x1,y1,x2,y2 in line:

            if x1 == x2:
                pass #Vertical Lines
            else:
                m = (y2 - y1) / (x2 - x1)
                c = y1 - m * x1

                if m < 0:
                    left_lines.append((m,c))
                elif m >= 0:
                    right_lines.append((m,c))

    if left_lines:
        left_line = np.mean(left_lines, axis=0)
    else:
        left_line = (0.001, 0)  # Default value if no lines found

    if right_lines:
        right_line = np.mean(right_lines, axis=0)
    else:
        right_line = (0.001, 0)  # Default value if no lines found

    print(left_line, right_line)

    for slope, intercept in [left_line, right_line]:

        #getting complete height of image in y1
        rows, cols = image.shape[:2]
        y1= int(rows) #image.shape[0]

        #taking y2 upto 60% of actual height or 60% of y1
        y2= int(rows*0.6) #int(0.6*y1)

        #we know that equation of line is y=mx +c so we can write it x=(y-c)/m

        x1=int((y1-intercept)/slope)
        x2=int((y2-intercept)/slope)
        poly_vertices.append((x1, y1))
        poly_vertices.append((x2, y2))
        draw_lines(img, np.array([[[x1,y1,x2,y2]]]))

    poly_vertices = [poly_vertices[i] for i in order]
    cv2.fillPoly(img, pts = np.array([poly_vertices],'int32'), color = (0,255,0))
    return cv2.addWeighted(image,0.7,img,0.4,0.)

    #cv2.polylines(img,np.array([poly_vertices],'int32'), True, (0,0,255), 10)
    #print(poly_vertices)


In [30]:
def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):

    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    line_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)

    line_img = assign_slopes_intercepts(line_img,lines)
    return line_img



def weighted_img(img, initial_img, α=0.1, β=1., γ=0.):

    lines_edges = cv2.addWeighted(initial_img, α, img, β, γ)

    return lines_edges




def quadrilateral_vertices(image):
    rows, cols = image.shape[:2]
    bottom_left  = [cols*0.15, rows]
    top_left     = [cols*0.45, rows*0.6]
    bottom_right = [cols*0.95, rows]
    top_right    = [cols*0.55, rows*0.6]

    ver = np.array([[bottom_left, top_left, top_right, bottom_right]], dtype=np.int32)
    return ver

In [31]:
def lane_finding_pipeline(image):

    #Grayscale
    gray_img = convert_to_gray(image)

    #Gaussian Smoothing
    smoothed_img = gaussian_blur(img = gray_img, kernel_size = 5)
    #Canny Edge Detection
    canny_img = edge_detection(img = smoothed_img, low_threshold = 180, high_threshold = 240)
    #Masked Image Within a Polygon
    masked_img = region_of_interest(img = canny_img, vertices = quadrilateral_vertices(image))
    #Hough Transform Lines
    houghed_lines = hough_lines(img = masked_img, rho = 1, theta = np.pi/180, threshold = 20, min_line_len = 20, max_line_gap = 180)
    #Draw lines on edges
    output = weighted_img(img = houghed_lines, initial_img = image, α=0.8, β=1., γ=0.)

    return output

In [32]:
!pip install moviepy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [33]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

In [34]:
white_output = './lanes_clip.mp4'
clip1 = VideoFileClip("test_videos/lanes_clip









                      .mp4")
white_clip = clip1.fl_image(lane_finding_pipeline) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)



t:  16%|█▋        | 79/485 [04:31<00:34, 11.75it/s, now=None]

t:  16%|█▋        | 79/485 [04:31<00:34, 11.75it/s, now=None]

(0.001, 0) [   0.97069416 -270.14171474]
Moviepy - Building video ./lanes_clip.mp4.
Moviepy - Writing video ./lanes_clip.mp4



t:   0%|          | 5/1297 [00:00<00:30, 42.04it/s, now=None]

(0.001, 0) [   0.97069416 -270.14171474]
(0.001, 0) [   0.95265674 -256.36628099]
(0.001, 0) [   0.95661128 -259.91039111]
(0.001, 0) [   0.95661128 -259.91039111]
(0.001, 0) [   0.95588309 -260.19769785]
(0.001, 0) [   0.9857695  -284.09783269]
(0.001, 0) [   0.9857695  -284.09783269]
(0.001, 0) [   0.95170976 -255.55033539]


t:   1%|          | 15/1297 [00:00<00:32, 39.52it/s, now=None]

...

(0.001, 0) [   1.01350267 -287.03663102]
(0.001, 0) [   1.01350267 -287.03663102]
(0.001, 0) [   1.01350267 -287.03663102]
(0.001, 0) [   1.01350267 -287.03663102]
(0.001, 0) [   1.01350267 -287.03663102]


  warnings.warn("Warning: in file %s, "%(self.filename)+

  warnings.warn("Warning: in file %s, "%(self.filename)+

t: 100%|██████████| 1297/1297 [01:14<00:00, 23.87it/s, now=None]WARNING:py.warnings:/usr/local/lib/python3.10/dist-packages/moviepy/video/io/ffmpeg_reader.py:123: UserWarning: Warning: in file test_videos/lanes_clip.mp4, 2764800 bytes wanted but 0 bytes read,at frame 1296/1298, at time 25.92/25.94 sec. Using the last valid frame instead.
  warnings.warn("Warning: in file %s, "%(self.filename)+



(0.001, 0) [   1.01350267 -287.03663102]
(0.001, 0) [   1.01350267 -287.03663102]
(0.001, 0) [   1.01350267 -287.03663102]
(0.001, 0) [   1.01350267 -287.03663102]




t:  16%|█▋        | 79/485 [05:46<00:34, 11.75it/s, now=None]

t:  16%|█▋        | 79/485 [05:46<00:34, 11.75it/s, now=None]

Moviepy - Done !
Moviepy - video ready ./lanes_clip.mp4
CPU times: user 24.6 s, sys: 4.66 s, total: 29.2 s
Wall time: 1min 15s
